# ARMAX Model Implementation with NumPy and SciPy


The code provided is an implementation of the ARMAX model, which is an extension of the ARMA model that includes exogenous variables. The ARMAX model combines autoregressive (AR) terms, moving average (MA) terms, and exogenous variables. The general form of an ARMAX(p, q) model can be expressed as:

$$y_t = c + \varphi_1y_{t-1} + \cdots + \varphi_py_{t-p} + \theta_1e_{t-1} + \cdots + \theta_qe_{t-q} + \beta_1x_{1,t} + \cdots + \beta_kx_{k,t} + e_t$$

where:
- $y_t$ is the value of the time series at time t
- $c$ is a constant term
- $\varphi_i$ are the AR coefficients for i = 1, ..., p
- $\theta_j$ are the MA coefficients for j = 1, ..., q
- $e_t$ is the error term (residual) at time t
- $x_{i,t}$ are the exogenous variables at time t
- $\beta_i$ are the coefficients for the exogenous variables
- $p$ is the order of the autoregressive part (AR)
- $q$ is the order of the moving average part (MA)

Now, let's break down the math behind each part of the code:

1. **armax_residuals function:**

The purpose of this function is to compute the residuals of the ARMAX model. Residuals are the differences between the observed values and the predicted values. The calculation is performed using the AR, MA, and exogenous coefficients:

- `ar_sum`: Calculates the sum of the autoregressive terms for the current time step.
- `ma_sum`: Calculates the sum of the moving average terms for the current time step.
- `exog_sum`: Calculates the sum of the exogenous terms for the current time step.

The residual at time `t` is calculated as:

$$\text{residuals}[t] = \text{data}[t] - \text{ar_sum} - \text{ma_sum} - \text{exog_sum}$$

2. **armax_predict function:**

This function predicts future values using the ARMAX model. The calculation is performed in a similar manner as in the `armax_residuals` function:

- `ar_sum`: Calculates the sum of the autoregressive terms for the current time step.
- `ma_sum`: Calculates the sum of the moving average terms for the current time step.
- `exog_sum`: Calculates the sum of the exogenous terms for the current time step.

The prediction at time `t` is calculated as:

$$\text{prediction} = \text{ar_sum} + \text{ma_sum} + \text{exog_sum}$$

3. **fit_armax function:**

This function estimates the AR, MA, and exogenous coefficients using the given data, exogenous variables, and the orders of the AR and MA components. It does this by minimizing the sum of squared residuals using the `minimize` function from the `scipy.optimize` library.

The objective function to be minimized is the sum of squared residuals:

$$\text{objective} = \sum (\text{armax_residuals}(\text{params}, \text{data}, \text{exog}, p, q))^2$$

The `minimize` function is used to find the optimal parameters that minimize the objective function:

$$\text{result} = \text{minimize}(\lambda \text{params}: \text{objective}, \text{init_params}, \text{method}='\text{L-BFGS-B}')$$

The `result.x` attribute of the `minimize` function's output contains the optimal AR, MA, and exogenous coefficients.



In [5]:
# Import the NumPy library, It's a fundamental library for scientific computing in Python.
import numpy as np

# Import the minimize function from the scipy.optimize module. This function is used for
# optimization tasks, such as finding the minimum value of a function or fitting parameters
# to a model. In this case, it will be used to estimate the AR, MA, and exogenous coefficients
# for the ARMAX model by minimizing the sum of squared residuals.

from scipy.optimize import minimize


In [3]:


def armax_residuals(params, data, exog, p, q):
    # Separate AR, MA, and exogenous coefficients from the input params array
    ar_params = params[:p]
    ma_params = params[p:p+q]
    exog_params = params[p+q:]

    # Get the length of the data and initialize an array of zeros for residuals
    n = len(data)
    residuals = np.zeros(n)

    # Loop through each data point
    for t in range(n):
        # Calculate the sum of autoregressive terms, if we have enough history
        ar_sum = np.sum(ar_params * data[t-p:t][::-1]) if t >= p else 0
        
        # Calculate the sum of moving average terms, if we have enough history
        ma_sum = np.sum(ma_params * residuals[t-q:t][::-1]) if t >= q else 0
        
        # Calculate the sum of exogenous terms using the dot product
        exog_sum = np.dot(exog_params, exog[t])
        
        # Compute the residual at time t
        residuals[t] = data[t] - ar_sum - ma_sum - exog_sum

    # Return the array of residuals
    return residuals


def armax_predict(params, data, exog, p, q, steps):
    # Separate AR, MA, and exogenous coefficients from the input params array
    ar_params = params[:p]
    ma_params = params[p:p+q]
    exog_params = params[p+q:]

    # Get the length of the data
    n = len(data)
    
    # Create a list of historical data points and residuals
    history = list(data)
    residuals = list(armax_residuals(params, data, exog, p, q))
    
    # Initialize an empty list to store the predictions
    predictions = []

    # Loop through the number of steps to predict
    for t in range(steps):
        # Calculate the sum of autoregressive terms using the historical data
        ar_sum = np.sum(ar_params * history[-p:][::-1]) if len(history) >= p else 0
        
        # Calculate the sum of moving average terms using the residuals
        ma_sum = np.sum(ma_params * residuals[-q:][::-1]) if len(residuals) >= q else 0
        
        # Calculate the sum of exogenous terms using the dot product
        exog_sum = np.dot(exog_params, exog[n+t]) if n+t < len(exog) else 0
        
        # Compute the prediction at the current time step
        prediction = ar_sum + ma_sum + exog_sum
        
        # Append the prediction to the list of predictions
        predictions.append(prediction)

        # Update the history and residuals with the new prediction
        history.append(prediction)
        residuals.append(0)  # Assuming zero residual for prediction

    # Return the list of predictions
    return predictions


def fit_armax(data, exog, p, q):
    # Initialize random AR, MA, and exogenous coefficients as the starting point for optimization
    init_params = np.random.normal(size=p+q+len(exog[0]))

    # Define the objective function for the optimization process:
    # The sum of squared residuals from the armax_residuals function
    objective = lambda params: np.sum(np.square(armax_residuals(params, data, exog, p, q)))

    # Optimize the objective function using the minimize function from scipy.optimize,
    # which finds the AR, MA, and exogenous coefficients that minimize the sum of squared residuals
    result = minimize(
        objective,         # The objective function to minimize
        init_params,       # The initial guess for the parameters
        method='L-BFGS-B', # The optimization algorithm (Limited-memory Broyden-Fletcher-Goldfarb-Shanno)
    )

    # Return the optimal AR, MA, and exogenous coefficients found by the optimization process
    return result.x





In [4]:
# Generate synthetic data 
np.random.seed(42)
n = 100
# Generate signal
data = np.random.normal(size=n)
# Generate exogenous variables
exog = np.random.normal(size=(n, 1)) 

# Fit the ARMAX model
p, q = 2, 2  # Specify the order of the AR and MA components
params = fit_armax(data, exog, p, q)
print("ARMAX parameters:", params)

# Make predictions
steps = 3  # Number of steps to predict
predictions = armax_predict(params, data, exog, p, q, steps)
# Print predictions
print("Predictions:", predictions)

ARMAX parameters: [ 0.71225548 -0.41436492 -0.76463476  0.415192   -0.15114309]
Predictions: [0.1119318973441556, 0.03351721918525502, -0.02250782841747838]
